In [1]:
%load_ext tensorboard

from headers import *
from common import CreateAndSaveModel
from common import GetTrainTestSets
from common import ClearTrainingOutput
 
def BuildModel(x_learn_layers = 564, learn_rate = 0.0001, l1 = None, l2 = None, top_layer_cnt = 1, units = 512, version = 0):
    basemodel = BaseModel(learn_rate = learn_rate, L1 = l1, L2 = l2,
                          top_layer_cnt = top_layer_cnt, units = units, version = version)
    tunner_model = ResNet152V2(include_top=False, weights='imagenet')
    return PredefinedModel(model = tunner_model, top_layers = x_learn_layers, intercept_module = basemodel).Create()

# 3 vs 4

In [2]:
train_set, test_set = GetTrainTestSets([[3],[4]], [[2048], [2048]], [[1024], [1024]])
keras_tuning_project_name = 'res_net_v_two'# throw exception when either capital letters or numbers are in name of directory
tunning_epoch_cnt = 5

Found 4096 validated image filenames belonging to 2 classes.
Found 2048 validated image filenames belonging to 2 classes.


## (3 vs 4): Keras Tuning 

In [ ]:
#put the weight information into the memory for each configuration and epoch
#so this method could consume tens of gigabays of memory
#devide one configuration into little ones 
def model_builder(hp): 
    hp_units = hp.Choice('units', values = [512,1024])
    hp_top_layer_cnt = hp.Choice('top_layer_cnt', values = [1,2])
    hp_l1 = hp.Choice('l1', values = [0.0, 0.0001, 0.0005, 0.001])
    hp_l2 = hp.Choice('l2', values = [0.0, 0.0001, 0.0005, 0.001])
    hp_learn_rate = hp.Choice('learn_rate', values = [0.001, 0.0001])
    predefinedModel = BuildModel(learn_rate = hp_learn_rate, l1 = hp_l1, l2 = hp_l2,
                          top_layer_cnt = hp_top_layer_cnt, units = hp_units)
    return predefinedModel.model

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = tunning_epoch_cnt,
                     factor = 2,
                     directory = 'keras_tuning', # throw exception when either capital letters or numbers are in name of directory
                     project_name = keras_tuning_project_name) # throw exception when either capital letters or numbers are in name of directory

In [ ]:
tuner.search(train_set, epochs = tunning_epoch_cnt, validation_data = test_set, callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
tuner.results_summary()
#mmodels = tuner.get_best_models(num_models = 3)

Summary:
1. Score: 0.66, L1 = L2 = LearnRate = 0.001
2. Score: 0.65, L1 = L2 = 0.0005, LearnRate = 0.001
3. Score: 0.65, L2 = 0.0005, LearnRate = 0.0001

## (3 vs 4): Tuning1

In [3]:
EPOCH = 10
predefinedModel = BuildModel(learn_rate = 0.001, l1 = 0.001, l2 = 0.001,top_layer_cnt = 1, units = 512)
extra_callbacks = [TensorBoard(log_dir="logs", histogram_freq=1)]

Model resnet152v2_TL564_PredefinedModel_F32_U512x1_LR001_L1001_L2001_LCnt6_V is built


In [ ]:
predefinedModel.fit(train_set = train_set, val_set = test_set, epochs = EPOCH,log = True, threshold = 0.65)

Train for 64 steps, validate for 16 steps
Epoch 1/10
64/64 [==============================] - ETA: 2:58:42 - loss: 1.7584 - accuracy: 0.359 - ETA: 2:11:26 - loss: 2.3499 - accuracy: 0.476 - ETA: 1:54:30 - loss: 2.2087 - accuracy: 0.489 - ETA: 1:44:08 - loss: 2.0284 - accuracy: 0.496 - ETA: 1:40:04 - loss: 1.9175 - accuracy: 0.512 - ETA: 1:34:57 - loss: 1.8421 - accuracy: 0.510 - ETA: 1:31:27 - loss: 1.7811 - accuracy: 0.522 - ETA: 1:27:47 - loss: 1.7347 - accuracy: 0.525 - ETA: 1:24:06 - loss: 1.7083 - accuracy: 0.519 - ETA: 1:21:01 - loss: 1.6772 - accuracy: 0.529 - ETA: 1:17:49 - loss: 1.6516 - accuracy: 0.532 - ETA: 1:15:18 - loss: 1.6286 - accuracy: 0.536 - ETA: 1:13:12 - loss: 1.6086 - accuracy: 0.534 - ETA: 1:11:11 - loss: 1.5887 - accuracy: 0.543 - ETA: 1:08:55 - loss: 1.5693 - accuracy: 0.547 - ETA: 1:07:05 - loss: 1.5534 - accuracy: 0.545 - ETA: 1:05:28 - loss: 1.5367 - accuracy: 0.551 - ETA: 1:03:55 - loss: 1.5235 - accuracy: 0.564 - ETA: 1:02:04 - loss: 1.5093 - accuracy: 0.

64/64 [==============================] - ETA: 2:30:32 - loss: 0.4451 - accuracy: 0.890 - ETA: 1:49:59 - loss: 0.4801 - accuracy: 0.867 - ETA: 1:36:09 - loss: 0.4886 - accuracy: 0.833 - ETA: 1:27:56 - loss: 0.4876 - accuracy: 0.843 - ETA: 1:24:01 - loss: 0.5243 - accuracy: 0.834 - ETA: 1:19:44 - loss: 0.5119 - accuracy: 0.835 - ETA: 1:16:34 - loss: 0.5359 - accuracy: 0.825 - ETA: 1:13:55 - loss: 0.5333 - accuracy: 0.820 - ETA: 1:11:17 - loss: 0.5774 - accuracy: 0.819 - ETA: 1:09:42 - loss: 0.5803 - accuracy: 0.818 - ETA: 1:07:47 - loss: 0.5679 - accuracy: 0.823 - ETA: 1:06:03 - loss: 0.5615 - accuracy: 0.824 - ETA: 1:04:48 - loss: 0.5558 - accuracy: 0.830 - ETA: 1:03:06 - loss: 0.5598 - accuracy: 0.824 - ETA: 1:01:41 - loss: 0.5547 - accuracy: 0.827 - ETA: 1:00:01 - loss: 0.5616 - accuracy: 0.828 - ETA: 58:34 - loss: 0.5565 - accuracy: 0.8309  - ETA: 57:44 - loss: 0.5501 - accuracy: 0.834 - ETA: 56:21 - loss: 0.5483 - accuracy: 0.833 - ETA: 55:09 - loss: 0.5461 - accuracy: 0.834 - ETA: 

64/64 [==============================] - ETA: 3:03:34 - loss: 0.1627 - accuracy: 0.968 - ETA: 2:23:43 - loss: 0.1604 - accuracy: 0.960 - ETA: 2:05:55 - loss: 0.2399 - accuracy: 0.921 - ETA: 1:55:14 - loss: 0.2439 - accuracy: 0.914 - ETA: 1:49:39 - loss: 0.2237 - accuracy: 0.918 - ETA: 1:45:14 - loss: 0.2174 - accuracy: 0.919 - ETA: 1:40:55 - loss: 0.2123 - accuracy: 0.928 - ETA: 1:37:10 - loss: 0.2137 - accuracy: 0.923 - ETA: 1:33:42 - loss: 0.2096 - accuracy: 0.923 - ETA: 1:31:30 - loss: 0.2188 - accuracy: 0.926 - ETA: 1:28:31 - loss: 0.2143 - accuracy: 0.930 - ETA: 1:25:28 - loss: 0.2092 - accuracy: 0.933 - ETA: 1:23:03 - loss: 0.2032 - accuracy: 0.938 - ETA: 1:20:24 - loss: 0.2023 - accuracy: 0.939 - ETA: 1:18:07 - loss: 0.1967 - accuracy: 0.942 - ETA: 1:16:36 - loss: 0.1928 - accuracy: 0.944 - ETA: 1:14:45 - loss: 0.1972 - accuracy: 0.943 - ETA: 1:13:51 - loss: 0.1974 - accuracy: 0.942 - ETA: 1:12:07 - loss: 0.1965 - accuracy: 0.944 - ETA: 1:10:13 - loss: 0.1933 - accuracy: 0.945 -

64/64 [==============================] - ETA: 3:50:04 - loss: 0.0706 - accuracy: 0.968 - ETA: 2:47:43 - loss: 0.0628 - accuracy: 0.976 - ETA: 2:25:48 - loss: 0.0568 - accuracy: 0.984 - ETA: 2:13:17 - loss: 0.0615 - accuracy: 0.984 - ETA: 2:05:49 - loss: 0.0623 - accuracy: 0.984 - ETA: 2:00:09 - loss: 0.0614 - accuracy: 0.984 - ETA: 1:55:16 - loss: 0.0578 - accuracy: 0.986 - ETA: 1:54:59 - loss: 0.0620 - accuracy: 0.986 - ETA: 1:52:25 - loss: 0.0611 - accuracy: 0.987 - ETA: 1:50:25 - loss: 0.0584 - accuracy: 0.989 - ETA: 1:48:06 - loss: 0.0579 - accuracy: 0.990 - ETA: 1:44:49 - loss: 0.0575 - accuracy: 0.990 - ETA: 1:41:55 - loss: 0.0584 - accuracy: 0.989 - ETA: 1:39:10 - loss: 0.0567 - accuracy: 0.990 - ETA: 1:36:50 - loss: 0.0564 - accuracy: 0.989 - ETA: 1:36:23 - loss: 0.0561 - accuracy: 0.989 - ETA: 1:36:09 - loss: 0.0552 - accuracy: 0.989 - ETA: 1:35:00 - loss: 0.0554 - accuracy: 0.988 - ETA: 1:32:59 - loss: 0.0552 - accuracy: 0.989 - ETA: 1:31:01 - loss: 0.0553 - accuracy: 0.988 -

64/64 [==============================] - ETA: 2:44:04 - loss: 0.0258 - accuracy: 1.000 - ETA: 2:02:38 - loss: 0.0440 - accuracy: 0.992 - ETA: 1:45:02 - loss: 0.0456 - accuracy: 0.989 - ETA: 1:35:19 - loss: 0.0469 - accuracy: 0.992 - ETA: 1:28:22 - loss: 0.0541 - accuracy: 0.984 - ETA: 1:24:53 - loss: 0.0483 - accuracy: 0.987 - ETA: 1:21:50 - loss: 0.0462 - accuracy: 0.988 - ETA: 1:18:18 - loss: 0.0436 - accuracy: 0.990 - ETA: 1:15:37 - loss: 0.0507 - accuracy: 0.986 - ETA: 1:13:46 - loss: 0.0504 - accuracy: 0.985 - ETA: 1:11:35 - loss: 0.0473 - accuracy: 0.987 - ETA: 1:09:42 - loss: 0.0470 - accuracy: 0.988 - ETA: 1:07:55 - loss: 0.0454 - accuracy: 0.989 - ETA: 1:06:21 - loss: 0.0453 - accuracy: 0.988 - ETA: 1:04:42 - loss: 0.0450 - accuracy: 0.989 - ETA: 1:02:58 - loss: 0.0453 - accuracy: 0.988 - ETA: 1:01:41 - loss: 0.0475 - accuracy: 0.986 - ETA: 1:00:27 - loss: 0.0493 - accuracy: 0.984 - ETA: 59:03 - loss: 0.0492 - accuracy: 0.9844  - ETA: 57:38 - loss: 0.0498 - accuracy: 0.983 - E

In [8]:
test_set

### (3 vs 4): Save Model1

In [ ]:
CreateAndSaveModel(root_dir = '', model = predefinedModel.model, test_set,
                   name = predefinedModel.getDescription(), extent = 'hdf5') "???"